In [1]:
pip install transformers[torch] pandas numpy datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 28.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 181.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 138.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np

In [6]:
def load_dataset_yelp():
    df =  pd.read_csv("yelp_academic_dataset_review.csv")
    dataset =  Dataset.from_pandas(df)
    return dataset.train_test_split(test_size=0.2)


In [19]:
def train_bert_model(dataset):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    def tokenize(example):
        return tokenizer(example["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize, batched=True)
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="no",
        logging_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        save_total_limit=1,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
    )
    trainer.train()
    # model.save_pretrained("./trained_bert")
    # tokenizer.save_pretrained("./trained_bert")
    return model.to("cuda" if torch.cuda.is_available() else "cpu"), tokenizer

# ---- Step 3: Inference Function ----
def analyze_sentiment(model, tokenizer, review_text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(review_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return ["Negative", "Neutral", "Positive"][prediction]


if __name__ == "__main__":
    dataset = load_dataset_yelp()
    model, tokenizer = train_bert_model(dataset)

    examples = [
        "I absolutely loved the pizza! Best in town.",
        "The service was okay, nothing special.",
        "Terrible experience. The food was cold and bland."
    ]

    for review in examples:
        sentiment = analyze_sentiment(model, tokenizer, review)
        print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
1000,0.620100
2000,0.352600


Review: I absolutely loved the pizza! Best in town.
Predicted Sentiment: Positive

Review: The service was okay, nothing special.
Predicted Sentiment: Neutral

Review: Terrible experience. The food was cold and bland.
Predicted Sentiment: Negative



In [21]:
torch.save(model.state_dict(), 'model_weights.pth')

In [22]:
tokenizer.save_pretrained(save_directory)

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json')